**PROJECT-2 (INDIVIDUAL)**


In this project, you will derive, implement, and demonstrate the error properties of a finite difference method for solving the heat equation

\begin{gather}
  \begin{aligned}
\frac{\partial u}{\partial t} = \kappa \frac{\partial^2 u}{\partial x^2} + g(x,t), \quad 0 \le t \le T, \; a \le x \le b
  \label{eqn:heateqn1D_def}
 \end{aligned}
 \tag{1}
\end{gather}


with $a=2$, $b=15$, $\kappa = 2$, and $T=5$. The $g(x,t)$ that is used for this problem is cumbersome to write down, so express this term generically as $g(x,t)$ in your analytical work. For the coding part of the problem, the exact $g(x,t)$ is provided in the skeleton code.

The initial and boundary conditions we will use to close the problem are
\begin{gather}
	u(x,t=0) = 0 
    \tag{2}
\end{gather}

\begin{gather}
	u(x=a,t) = 0 
    \tag{3}
\end{gather}

\begin{gather}
	u(x=b,t) = 0
    \tag{4}
\end{gather}


1.)   Derive the first step in the method of lines for $p=2$. That is, use a second-order spatial representation to simplify the IBVP into an initial value problem. Be sure to account for boundary conditions appropriately, so that you arrive at a system $\dot{\boldsymbol{u}} = \boldsymbol{Au} + \boldsymbol{g}$. You should write out the matrix $\boldsymbol{A}$ in detail, specifying how you are accounting for the boundary conditions. The vector $\boldsymbol{g}$ should be written in terms of $g(x_j,t)$, for appropriate indices $j$.

2.)   Let's say that we choose to solve the resulting IVP using a trapezoid method. Starting from the initial value problem you obtained in (1), describe how you will solve the IVP using the trapezoid method. You need not derive the trapezoid method explicitly, but write out the algorithm and describe in words how you will obtain an approximate solution to the heat equation at some instance $t_{k+1}$.

3.)   Write a code that implements the algorithm you developed in parts (1) and (2). For the specific $g(x,t)$ provided in the skeleton code, the exact solution is 
    
\begin{gather}
			u_{e} = \sin\left[ t \sin\left(\frac{6\pi(x-a)}{b-a}\right) \right]
\tag{5}
\end{gather}
	
    
   In this part of the problem, you will do a spatial convergence test to demonstrate that your algorithm scales as $O(\Delta x^2)$. Using a fixed small $\Delta t$ of $\Delta t = 10^{-3}$, run your code for $n=[20, 40, 80, 120, 240]^T$. The reason for running for a fixed small value of $\Delta t$ is that it ensures that the temporal error contribution is small, so the spatial error that we want to investigate is dominant. For this question, plot the following:
	
   (a) Create a waterfall plot for the exact and finite difference solutions for the finest value of $n$. This will build intuition and give a qualitative, ``eyeball-norm'' demonstration that the numerical and exact solutions agree. 
   
   (b) Now demonstrate the correctness of your method quantitatively. Plot the error of your method at $T=5$ and show that the convergence rate is $O(\Delta x^2)$.
	 
	
4.) Modify your code from 3 to do a temporal convergence test. For a large fixed value of $n=3000$ (or equivalently, a small fixed value of $\Delta x$), plot the error in your numerical solution at $T=5$ for $\Delta t = [2.5, 1, 0.5, 0.25, 0.1]^T$. You should see the error scale as $O(\Delta t^2)$. Again, we set $\Delta x$ to be small so that the spatial error does not interfere with our investigation of the convergence behavior in the temporal error. There is no need to provide a waterfall plot for this part of the problem.



# Question 1

This part is going to fuck me

# Question 2

Starting from the result of Question 1, we can use the trapezoid method to solve for the values of $u$. This begins with using the trapezoidal function:

\begin{equation}
    u_{k+1} = u_k + \dfrac{\Delta t}{2} (f(u_k,t_k) + f(u_{k+1},t_{k+1}))
\end{equation}
where $f = Au + g(t)$
Using this relation, we can recast the trapezoidal function as
\begin{equation}
    u_{k+1} = u_k + \dfrac{\Delta t}{2} ((Au_k,+g(t_k)) + (Au_{k+1} + g(t_{k+1})))
\end{equation}
\begin{equation}
    \rightarrow (I - \dfrac{\Delta t}{2}A) u_{k+1} = u_k + \dfrac{\Delta t}{2} ((Au_k,+ g(t_k)) + (g(t_{k+1})))
\end{equation}
\begin{equation}
    \rightarrow u_{k+1} = (I - \dfrac{\Delta t}{2}A)^{-1}[u_k + \dfrac{\Delta t}{2} [Au_k+ g(t_k) + g(t_{k+1})]]
\end{equation}

Then, using the boundary conditions given in the problem, (namely that $u(x,t=0) = 0$), the system can be advanced in time to a time $t_{k+1}$ to obtain an approximate solution to the heat equation. 

# Question 3

In [1]:
import sympy
import numpy as np
from numpy import sin, cos, pi
from numpy import linalg
import matplotlib.pyplot as plt
#start with basic variables
T = 5
a = 2
b = 15
L = b-a
kappa = 2
dt = 0.001
tlist = np.arange(0,T+dt,dt)

nlist = [20,40,80,120,240]
#exact g(x,t) using uexa(the exact solution)
def g(X,T):
    g = 72*pi**2*T**2*sin(T*sin(pi*(6*X/13 - 12/13)))*cos(pi*(6*X/13 - 12/13))**2/169 + 72*pi**2*T*sin(pi*(6*X/13 - 12/13))*cos(T*sin(pi*(6*X/13 - 12/13)))/169 + sin(pi*(6*X/13 - 12/13))*cos(T*sin(pi*(6*X/13 - 12/13)))
    return g

#define uexact function
def uexact(x,L,t):
    return np.sin(t*np.sin(6*np.pi*(x-a)/(b-a)))
#implement BCs
alpha = 0
beta = 0

#create the code
def heat(n):
    xj = np.zeros(n+1)
    for i in range(n+1):
        xj[i] = a + ((b-a)*(i-1))/n
    #spacing
    dx = xj[1]-xj[0]
    #build A
    A = np.diag(-2*np.ones(n-1)) + np.diag(np.ones(n-2),k=1) + np.diag(np.ones(n-2),k=-1)
    A = kappa/(dx**2) * A

    gv = np.zeros((len(tlist),n-1))
    u = np.zeros((len(tlist),n-1))
    for x in range(n-1):
        for t in range(len(tlist)):
            gv[t][x] = g(xj[x+2],tlist[t])
        for j in range(len(tlist)-1):
            u[j+1] = linalg.inv(np.eye(len(A)) - (dt/2)*A) @ (u[j] + (dt/2)*(A@u[j] + gv[j] + gv[j+1]))
    u_exact = np.zeros((len(tlist),n+1))
    u = np.block(([[np.zeros((len(tlist),1)),u,np.zeros((len(tlist),1))]]))
    #now for the exact
    for i in range(n):
        for j in range(len(tlist)):
            u_exact[j][i] = uexact(xj[i+1],L, tlist[j])
    err = np.linalg.norm(u[-1]-u_exact[-1]) / np.linalg.norm(u_exact[-1])
    
    return u, u_exact,err,xj

In [ ]:
#plotting
#simulating at each n    
u_list = []
u_true_list = []
err_list = []
x_list = []
for i in nlist:
    u, u_exact, err, x = heat(i)
    u_list.append(u[-1])
    u_true_list.append(u_exact[-1])
    err_list.append(err)
    x_list.append(x) 

In [ ]:
#plotting
fig = plt.figure(figsize=(16, 12), tight_layout=True)
ax1 = fig.add_subplot(231)
ax1.plot(x_list[0], u_list[0], 'k-', label='$u$')
ax1.plot(x_list[0], u_true_list[0], 'r--', label='$u_(true)$')
ax1.set_xlabel('$x$')
ax1.set_ylabel('')
ax1.set_title('$n$ = 20')
ax1.legend()

ax2 = fig.add_subplot(232)
ax2.plot(x_list[1], u_list[1], 'k-', label='$u$')
ax2.plot(x_list[1], u_true_list[1], 'r--', label='$u_(true)$')
ax2.set_xlabel('$x$')
ax2.set_ylabel('')
ax2.set_title('$n$ = 40')
ax2.legend()

ax3 = fig.add_subplot(233)
ax3.plot(x_list[2], u_list[2], 'k-', label='$u$')
ax3.plot(x_list[2], u_true_list[2], 'r--', label='$u_(true)$')
ax3.set_xlabel('$x$')
ax3.set_ylabel('')
ax3.set_title('$n$ = 80')
ax3.legend()

ax4 = fig.add_subplot(234)
ax4.plot(x_list[3], u_list[3], 'k-', label='$u$')
ax4.plot(x_list[3], u_true_list[3], 'r--', label='$u_(true)$')
ax4.set_xlabel('$x$')
ax4.set_ylabel('')
ax4.set_title('$n$ = 120')
ax4.legend()

ax5 = fig.add_subplot(235)
ax5.plot(x_list[4], u_list[4], 'k-', label='$u$')
ax5.plot(x_list[4], u_true_list[4], 'r--', label='$u_(true)$')
ax5.set_xlabel('$x$')
ax5.set_ylabel('')
ax5.set_title('$n$ = 240')
ax5.legend()    
    
#plotting 
c2 = err_list[-1]
c1 = c2*((nlist[-1])/(nlist[0]))**2
n_scaling = [nlist[0], nlist[-1]]
err_scaling = [c1, c2]

fig = plt.figure(figsize=(4, 3), tight_layout=True)
ax = fig.add_subplot(111)
ax.plot(n_scaling, err_scaling, 'k-', label='$O(\Delta x^2)$')
ax.plot(nlist, err_list, 'r.', label = '||e||_2')
### YOUR CODE HERE ###
ax.set_xlabel('$n$')
ax.set_ylabel('$||e||_2$')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xticks([10, 100])
ax.legend()

# Question 4